In [1]:
import numpy as np
import math

def TropMulti(A, B):

    m, n = A.shape
    k, l = B.shape

    if n != k:
        raise ValueError("Dimension Error")


    C = np.zeros((m, l))

    for i in range(m):
        for j in range(l):
            Ax1 = A[i, :] + B[:, j]
            C[i, j] = np.max(Ax1)

    return C

def maxplusId(n):
    A = -np.inf * np.ones((n, n))
    np.fill_diagonal(A, 0)

    return A

def GenerateRandomMatrixLindePuente(n, k, a):
    result = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                result[i, j] = k
            else:
                result[i, j] = np.random.randint(2*a, (1*a)+1)
    return result


def get_generatorLinde(alpha, n):
    matrix = -np.inf * np.ones((n, n))

    col = alpha // n
    row = alpha % n
    matrix[row, col] = 0
    return matrix





def GenerateKeyStickelsLinde(n):
    Wm=np.random.randint(-100, 101, size=(n, n))
    k1 = np.random.randint(0, 101)
    k2 = np.random.randint(0, 101)
    a = np.random.randint(-101, 1)
    b = np.random.randint(-101, 1)
    A1 = GenerateRandomMatrixLindePuente(n, k1, a)
    A2 = GenerateRandomMatrixLindePuente(n, k2, b)

    l1 = np.random.randint(0, 101)
    l2 = np.random.randint(0, 101)
    c = np.random.randint(-101, 1)
    d = np.random.randint(-101, 1)
    B1 = GenerateRandomMatrixLindePuente(n, l1, c)
    B2 = GenerateRandomMatrixLindePuente(n, l2, d)

    temp1 = TropMulti(A1, Wm)
    U=TropMulti(temp1,A2)

    temp2 = TropMulti(B1, Wm)
    V = TropMulti(temp2, B2)

    KA = TropMulti(A1, TropMulti(V, A2))
    KB = TropMulti(B1, TropMulti(U, B2))

    if np.array_equal(KA, KB):
        key = KA
    else:
        key = []

    return key, U, V,Wm


In [2]:
# part of this code is from https://github.com/mkotov/tropical3/blob/main/attack.py

import scipy.optimize
import heapq
import multiprocessing



def get_compressed_covers(F):


    def compress(G):


        def find(H, S):
            for i in range(len(H)):
                if S.inds == H[i].inds:
                    return i
            return None

        H = []
        for S in G:
            i = find(H, S)
            if i is None:
                H.append(S)
            else:
                H[i].ijs.update(S.ijs)
        return H

    def unite_sets(ss):

        return set() if len(ss) == 0 else set.union(*ss)

    def get_sets_with_unique_elements(Z):

        return list(filter(lambda S: len(S.inds.difference(unite_sets([T.inds for T in filter(lambda T: S != T, Z)]))) != 0, Z))

    def get_sets_without_elements(Z, N):

        return list(filter(lambda S: len(S.inds) != 0, [Cover(S.inds.difference(N), S.ijs) for S in Z]))

    if len(F) == 0:
        return [[]]
    Z = compress(F)
    M = get_sets_with_unique_elements(Z)
    P = get_sets_without_elements(Z, unite_sets([S.inds for S in M]))
    if len(P) == 0:
        return [M]

    P.sort(key=lambda S: len(S.inds), reverse=True)

    X = [[P[0]] + S for S in get_compressed_covers(
        get_sets_without_elements(P[1:], P[0].inds))]
    Y = get_compressed_covers(P[1:])

    return [M + S for S in X + Y]


def compute_preweights(S, R):
    lins = {}
    cols = {}
    for T in S:
        for p in T.ijs:
            i = p[0]
            j = p[1]
            if not i in lins:
                lins[i] = 0
            if not j in cols:
                cols[j] = 0
            if T != R:
                lins[i] += 1.0 / len(T.ijs)
                cols[j] += 1.0 / len(T.ijs)
    return lins, cols


def get_weighted_sets(S, solve_linprog):
    W = []
    mandatory = [next(iter(T.ijs))
                 for T in filter(lambda T: len(T.ijs) == 1, S)]

    for T in S:
        lins, cols = compute_preweights(S, T)
        if len(T.ijs) > 1:
            w = []
            for p in T.ijs:
                if solve_linprog(mandatory + [p]):
                    w.append([p, (lins[p[0]] + 1) * (cols[p[1]] + 1)])
            W.append([p[0]
                     for p in sorted(w, reverse=True, key=lambda x: x[1])])
        else:
            W.append([next(iter(T.ijs))])

    return W


def enumerate_with_queue(E, chunk_size=10):


    def heuristics_to_sort(S):
        def sum_of_cross(S, i, j):
            lins = sum(1 if s[0] == i else 0 for s in S)
            cols = sum(1 if s[1] == j else 0 for s in S)
            return lins * cols

        return -sum(sum_of_cross(S, s[0], s[1])**2 for s in S)

    q = []
    while True:
        k = 0
        for e in E:
            heapq.heappush(q, (heuristics_to_sort(e), e))
            k += 1
            if k == chunk_size:
                break

        if len(q) == 0:
            break

        r = heapq.heappop(q)
        yield r[1]


def enumerate_product_of_sets(W):

    def enumerate_product_of_sets_(W, s, i):
        if i == len(W) - 1:
            if s < len(W[i]):
                yield [W[i][s]]
            else:
                return
        else:
            for t in range(min(s + 1, len(W[i]))):
                for q in enumerate_product_of_sets_(W, s - t, i + 1):
                    yield [W[i][t]] + q

    l = sum(len(w) - 1 for w in W) + 1
    for s in range(l):
        yield from enumerate_product_of_sets_(W, s, 0)


class Cover:
    def __init__(self, inds, ijs):
        self.inds = inds
        self.ijs = ijs


def apply_attack(U,Wm):


    M = dict()
    I = []
    n = U.shape[0] * U.shape[1]
    for alpha in range(n):
        for beta in range(n):
            tempp=TropMulti(get_generatorLinde(alpha, int(math.sqrt(n))),Wm)
            T= TropMulti(tempp, get_generatorLinde(beta, int(math.sqrt(n)))) - U


            c_ab = np.min(-1 * T.flatten())

            if c_ab >= 20000:
              S_ab = np.zeros_like(T)
            else:
              S_ab = (-1 * T == c_ab)


            M[(alpha, beta)] = c_ab

            inds_t=np.argwhere(S_ab==1)

            inds = {(index[0], index[1]) for index in inds_t}

            I.append(Cover(inds, {(alpha, beta)}))

    G = get_compressed_covers(I)

    def solve_linprog(S):


        def make_matrices_for_linprog(S):

            c = [0 for _ in range(n + n+4)]
            Ain = []
            bin = []
            Aeq = []
            beq = []

            for i in range(n):
                for j in range(n):
                    v = [1 if k == i or k == n +
                         j else 0 for k in range(n + n+4)]
                    m = M[(i, j)]
                    if (i, j) in S:
                        Aeq.append(v)
                        beq.append(m)
                    else:
                      if m==np.inf:
                        m=10000
                      Ain.append(v)
                      bin.append(m)


            for i in range(int(math.sqrt(n))):
              for j in range(int(math.sqrt(n))):
                if i==j:
                  Aeq.extend([[0 for k in range(n+n+4)] for _ in range(1)])
                  Aeq[-1][i*int(math.sqrt(n))+j] = 1
                  Aeq[-1][n+n+2] = -1
                  beq.append(0)
                else:
                  Ain.extend([[0 for k in range(n+n+4)] for _ in range(2)])
                  Ain[-2][i*int(math.sqrt(n))+j]=-1
                  Ain[-2][n+n]=2
                  Ain[-1][i*int(math.sqrt(n))+j]=1
                  Ain[-1][n+n]=-1
                  bin.append(0)
                  bin.append(0)


            for i in range(int(math.sqrt(n))):
              for j in range(int(math.sqrt(n))):
                if i==j:
                  Aeq.extend([[0 for k in range(n+n+4)] for _ in range(1)])
                  Aeq[-1][n+i*int(math.sqrt(n))+j] = 1
                  Aeq[-1][n+n+3] = -1
                  beq.append(0)
                else:
                  Ain.extend([[0 for k in range(n+n+4)] for _ in range(2)])
                  Ain[-2][n+i*int(math.sqrt(n))+j]=-1
                  Ain[-2][n+n+1]=2
                  Ain[-1][n+i*int(math.sqrt(n))+j]=1
                  Ain[-1][n+n+1]=-1
                  bin.append(0)
                  bin.append(0)




            if Ain == []:
                Ain = None
                bin = None
            if Aeq == []:
                Aeq = None
                beq = None

            return c, Ain, bin, Aeq, beq

        c, Ain, bin, Aeq, beq = make_matrices_for_linprog(S)

        put_bounds = [(None, None)] * (2 * n) + [(-np.inf, 0)] * (2)+ [(0, np.inf)] * (2)
        T = scipy.optimize.linprog(
            c, A_ub=Ain, b_ub=bin, A_eq=Aeq, b_eq=beq, bounds=put_bounds)
        if T.success:
            return [[T.x[i] for i in range(n)], [T.x[n + i] for i in range(n)]]

    def enumerate_covers(G):


        for S in sorted(G, key=len):
            W = get_weighted_sets(S, solve_linprog)
            yield from enumerate_with_queue(enumerate_product_of_sets(W))

    i=0
    print(sum(len(sublist) for sublist in enumerate_covers(G)))
    for S in enumerate_covers(G):
        i=i+1
        result = solve_linprog(S)
        if result:
            print('tested covers',i)
            return result

    return None

In [ ]:
import time
import matplotlib.pyplot as plt
def Attack_time(min_dimension, max_dimension):
  times = []
  dimension = range(min_dimension, max_dimension + 1)
  for d in dimension:
    [key,U,V,Wm]=GenerateKeyStickelsLinde(d)
    start_time = time.time()
    res=apply_attack(U,Wm)
    x=res[0]
    y=res[1]
    x_reshaped = np.reshape(x, (d, d))
    y_reshaped = np.reshape(y, (d, d))
    x_TropMulti_V = TropMulti(np.transpose(x_reshaped), V)
    K_attack = TropMulti(x_TropMulti_V, np.transpose(y_reshaped))
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)

  plt.plot(dimension, times, marker='o')
  plt.title('Excution time vs Dimension')
  plt.xlabel('Dimension n')
  plt.ylabel('Excution time (seconds)')
  plt.grid(True)
  plt.show()


Attack_time(min_dimension=2, max_dimension=6)
